<a href="https://colab.research.google.com/github/QuinnStraus/CSCI-1470-final/blob/main/DL_Final_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
import os

In [3]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [5]:
STOP_TOKEN = '<stop>'
PAD_TOKEN = '<pad>'

WINDOW_SZ = 200


In [6]:
# Convert Text to array

text = []
root_dir = 'gdrive/Shared drives/CSCI1470 Final Project'
with open(os.path.join(root_dir, 'songs.txt'), 'r') as data_file:
  for line in data_file:
    text.append(line.split())


In [30]:
# Pad songs

padded_songs = []
for line in text:
  if len(line) != 0:
    song = line[:WINDOW_SZ]
    song += [STOP_TOKEN] + [PAD_TOKEN] * (WINDOW_SZ - len(song)-1)
    padded_songs.append(song)
print([len(padded_songs[i])for i in range(9)])
print(padded_songs[7])

[201, 201, 201, 201, 201, 201, 201, 201, 201]
['G#3d4/3', 'br', 'C4d1.5', 'E-4d1.5', 'G#4d1.5', 'br', 'F2d1.25', 'br', 'G#2d1.0', 'br', 'C3d0.75', 'br', 'E-3d1.0', 'br', 'G#3d2/3', 'br', 'C4d0.25', 'br', 'E-4d0.0', 'br', 'F3d1/3', 'br', 'C#4d4/3', 'br', 'F4d2.25', 'G#4d2.25', 'br', 'C#5d1.75', 'br', 'B-2d2.0', 'br', 'C#3d1.5', 'br', 'F3d1.0', 'br', 'G#3d1.0', 'br', 'C#4d1.0', 'br', 'F3d0.75', 'br', 'G#3d1/3', 'G#4d1/3', 'br', 'B-2d1/3', 'br', 'E-2d0.5', 'br', 'G4d2/3', 'br', 'B-2d2/3', 'br', 'G3d0.75', 'br', 'E-4d1.0', 'br', 'G4d1/3', 'br', 'E-3d2/3', 'br', 'D4d0.5', 'G4d0.5', 'br', 'E-3d0.5', 'br', 'C#4d0.25', 'G4d0.25', 'br', 'E-3d0.5', 'br', 'G#2d1.5', 'br', 'C4d1.75', 'br', 'G4d1.75', 'br', 'E-3d1.75', 'br', 'C5d2.5', 'br', 'G3d2.5', 'br', 'C4d0.75', 'br', 'E-3d1/3', 'br', 'E-3d0.75', 'br', 'G4d1/3', 'br', 'C#2d0.75', 'br', 'F4d0.75', 'br', 'G#2d0.75', 'br', 'G#3d1/3', 'F4d1/3', 'br', 'C#4d0.5', 'br', 'C#3d0.25', 'br', 'C#4d1/3', 'br', 'G#3d0.5', 'br', 'F4d0.25', 'br', 'F3d0.25', '

In [31]:
# Create vocab

tokens = []
for s in padded_songs: tokens.extend(s)
all_words = sorted(list(set([STOP_TOKEN,PAD_TOKEN] + tokens)))

vocab =  {word:i for i,word in enumerate(all_words)}


In [35]:
song_ids = np.stack([[vocab[word] for word in song] for song in padded_songs])
print(song_ids)

[[165 170 241 ...  25 241   1]
 [ 64 159 241 ...  64 159   1]
 [155 241 155 ... 155 241   1]
 ...
 [ 75 241 108 ... 226 241   1]
 [207 241  79 ... 218 241   1]
 [155 241 164 ... 158 162   1]]


In [ ]:
class Transformer_Seq2Seq(tf.keras.Model):
	def __init__(self, window_sz, vocab_sz):

		######vvv DO NOT CHANGE vvv##################
		super(Transformer_Seq2Seq, self).__init__()

		self.vocab_sz = vocab_sz
		self.window_sz = window_sz 

		# TODO:
		# 1) Define any hyperparameters
		# 2) Define embeddings, encoder, decoder, and feed forward layers

		# Define batch size and optimizer/learning rate
		self.batch_size = 100
		self.embedding_size = 32
		self.learning_rate = 0.001
		self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)

		# Define english and french embedding layers:
		self.fr_embedding = tf.keras.layers.Embedding(french_vocab_size, self.embedding_size)
		self.en_embedding = tf.keras.layers.Embedding(english_vocab_size, self.embedding_size)
		# Create positional encoder layers
		self.position_fr = transformer.Position_Encoding_Layer(self.french_window_size, self.embedding_size)
		self.position_en = transformer.Position_Encoding_Layer(self.english_window_size, self.embedding_size)
		# Define encoder and decoder layers:
		self.encoder = transformer.Transformer_Block(self.embedding_size, False)
		self.decoder = transformer.Transformer_Block(self.embedding_size, True)
		# Define dense layer(s)
		self.dense2 = tf.keras.layers.Dense(self.english_vocab_size, activation='softmax')

	@tf.function
	def call(self, encoder_input, decoder_input):
		"""
		:param encoder_input: batched ids corresponding to french sentences
		:param decoder_input: batched ids corresponding to english sentences
		:return prbs: The 3d probabilities as a tensor, [batch_size x window_size x english_vocab_size]
		"""
	
		# TODO:
		#1) Add the positional embeddings to french sentence embeddings
		located_french = self.position_fr(self.fr_embedding(encoder_input))
		#2) Pass the french sentence embeddings to the encoder
		encoded = self.encoder(located_french)
		#3) Add positional embeddings to the english sentence embeddings
		located_english = self.position_en(self.en_embedding(decoder_input))
		#4) Pass the english embeddings and output of your encoder, to the decoder
		decoded = self.decoder(located_english, encoded)
		#5) Apply dense layer(s) to the decoder out to generate probabilities
		probabilities = self.dense2(decoded)
		return probabilities
